In [ ]:
import thinkplot
import thinkstats2
import pandas as pd
import numpy as np
import scipy.stats as ss
from fractions import Fraction

##Seaborn for fancy plots. 
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams["figure.figsize"] = (15,5)

# Bayes and Updates

We will solve the Monty Hall problem, but first we can work on some more simple examples and build out a method for using Bayes probability calculations. Remember the Bayes theorem from before:

$$P(A|B) = \frac{P(A) P(B|A)}{P(B)}$$

Suppose you have two high school classes of 40 students - class A and class B. Each class has some failing students:

- Class A has 10 failing students, 30 passing ones. 

- Class B has 20 failing students, 20 passing ones.

If we randomly select one failing student, what is the probability they are from Class A. 

We can calculate this out as a table. First, another way to think of Bayes....

## Diachronic Bayes

There is another way to think of Bayes's theorem: it gives us a way to
update the probability of a hypothesis, $H$, given some body of data, $D$.

This interpretation is "diachronic", which means "related to change over time"; in this case, the probability of the hypotheses changes as we see new data.

Rewriting Bayes's theorem with $H$ and $D$ yields:

$$P(H|D) = \frac{P(H)~P(D|H)}{P(D)}$$

In this interpretation, each term has a name:

-  $P(H)$ is the probability of the hypothesis before we see the data, called the prior probability, or just **prior**.

-  $P(H|D)$ is the probability of the hypothesis after we see the data, called the **posterior**.

-  $P(D|H)$ is the probability of the data under the hypothesis, called the **likelihood**.

-  $P(D)$ is the **total probability of the data**, under any hypothesis.

Sometimes we can compute the prior based on background information. For example, the classroom problem specifies that we choose a student at random with equal probability.

In other cases the prior is subjective; that is, reasonable people might disagree, either because they use different background information or because they interpret the same information differently.

In [2]:
#Class A - 10/40 fails
#Class B - 20/40 fails
#Grab a rando fail, how likely is it that they are from A

table = pd.DataFrame(index=['Class A', 'Class B'])
table

""
Class A
Class B


We can then add in the first part - the prior probability. 

In [3]:
table['prior'] = 1/2, 1/2
table

,prior
Class A,0.5
Class B,0.5


Add the likelihoods in...
For class A there is a 25% chance of a student failing.
For class B, it is 50%

We can also think of the likelihoods explicitly as conditional statements:
<ul>
<li>E.g. "If I choose class B, what is the likelihood of getting a failiure?"
<li>Or, give the prior is true, now what is the probability?

You are assuming the "Question part" of the original goal - what are the chances this class provides a failiure, given the

In [4]:
#If I choose A, 10 out of 40 are failing, so the chances are 1/4
#If I choose B, 20 out of 40 are failing, so the chances are 1/2
table['likelihood'] = 1/4, 1/2
table

,prior,likelihood
Class A,0.5,0.25
Class B,0.5,0.50


Next, multiply the two probabilities together:
<ul>
<li>E.g. There's a 50% chance of choosing class B, and if I do, there's a 50% chance of getting a fail. 
</ul>
We label this column the unnorm - or unnormalized probabilities. This is because they are both accurate probabilities, but they are not normalized - they do not sum to 1. If we look a little closer, they are also part our boy Bayes' Theorem:
<ul>
<li>The numerator of Bayes is a probability multiplied by a conditional, which is the unnorm value.
<li>The denomenator is the total probability - there are only 2 cases here, one must be true, so it is the sum of the unnorms.

In [5]:
#Calculate unnormalized probabilities
table['unnorm'] = table['prior'] * table['likelihood']
table


,prior,likelihood,unnorm
Class A,0.5,0.25,0.125
Class B,0.5,0.50,0.250


As a check, we can demonstrate that last point is true:

- Calculate the total probability of getting a fail by summing the unnorms.

- Calculate the total probability of getting a fail by direct calculation. 

In [6]:
prob_data = table['unnorm'].sum()
print("Unnorms:", prob_data)
probDirect = (10+20)/(40+40) #The overall fail chances - 30 total failiures, 80 total students. 
print("Direct:", probDirect)

Unnorms: 0.375
Direct: 0.375


Now we can normalize - or make the probs total to 1. We just divide by that total probability. This gives us the posterior probabilities, answering our original question:

- P(Class A | Failing)

As well as giving us the other probabilites, for free. 

In [7]:
table['posterior'] = table['unnorm'] / prob_data
table

,prior,likelihood,unnorm,posterior
Class A,0.5,0.25,0.125,0.333333
Class B,0.5,0.50,0.250,0.666667


We can wrap those last steps un into a formula, since they are just calculating from the probabilites that we've provided.

In [8]:
def update(table):
    """Compute the posterior probabilities."""
    table['unnorm'] = table['prior'] * table['likelihood']
    prob_data = table['unnorm'].sum()
    table['posterior'] = table['unnorm'] / prob_data
    return prob_data

## Dice problem

Suppose I have a box with a 6-sided die, an 8-sided die, and a 12-sided die. I choose one of the dice at random, roll it, and report that the outcome is a 1. What is the probability that I chose the 6-sided die?

In [ ]:
#Create the table
dice = pd.DataFrame(index=["Six Side", "Eight Side", "12 Side"])
dice["prior"] = Fraction(1,3)
dice

In [ ]:
#The probability for each die being a 1, given I pick it.
dice["likelihood"] = Fraction(1, 6), Fraction(1, 8), Fraction(1, 12)
dice

In [ ]:
#Update to finish
update(dice)
dice


## The Monty Hall Problem

Next we'll use a Bayes table to solve one of the most contentious problems in probability.

The Monty Hall problem is based on a game show called *Let's Make a Deal*. If you are a contestant on the show, here's how the game works:

* The host, Monty Hall, shows you three closed doors -- numbered 1, 2, and 3 -- and tells you that there is a prize behind each door.

* One prize is valuable (traditionally a car), the other two are less valuable (traditionally goats).

* The object of the game is to guess which door has the car. If you guess right, you get to keep the car.

The key - after you pick a door, Monty will open another, revealing a goat. Then Monty offers you the option to stick with your original choice or switch to the remaining unopened door.

To maximize your chance of winning the car, should you stick with Door 1 or switch to Door 2?

To answer this question, we have to make some assumptions about the behavior of the host:

1.  Monty always opens a door and offers you the option to switch.

2.  He never opens the door you picked or the door with the car.

3.  If you choose the door with the car, he chooses one of the other
    doors at random.

In [ ]:
#Start off - initially the chances are equal for each door. 
#So the prior probabilities are all 1/3
monty = pd.DataFrame(index=["Door 1", "Door 2", "Door 3"])
monty["prior"] = Fraction(1,3)
monty

Now we need to decide what door we want - let's get that whip. 

<b>We'll assume we pick door 1.</b>

Then Monty opens one of the other doors, to us it is random. When he does so it gives us the likelihoods.

<b>We'll assume he opens door 3 - remember he always opens a goat door, not the car</b>

Now, that we know that it isn't door 3 (that's open, it is a goat). Remember, each one is a hypothetical - if we are in this "class" (door choice), what is the probability of "success" (a car there)?: 


We can think about this by carefully defining the problem - What are the odds that Monty opened Door 3, given that the Car is in Door X:
<ul>
<li>The likelihood he'd open Door 3 if the car is there is 0 - we can see the goat, and that's the rules. 
<li>The likelihood he'd open Door 3 if the car is in Door 2 is 1 - he'd be forced to by the rules of the game, you picked Door 1, Door 2 has the car, so he can only open Door 3.
<li>The linkelihood he'd open Door 3 if the car is in Door 1 is 1/2 - he just picks randomly 
</ul>

key - we don't really know the probability the car is in Door X directly. We can use the probability that the Door is opened, and the rules of the game to calculate it. 

In [ ]:
#We can update the table
monty["likelihood"] = Fraction(1,2),1,0
monty

In [ ]:
update(monty)
monty

Showing it mathmatically requires a bunch of derivation: https://en.wikipedia.org/wiki/Monty_Hall_problem

Alternate explaination that I think is the most clear way to imagine it: Initially there is a 1/3 chance of the car being behind each door. However, after you choose those odds change, due to the rules of the game:
<ul>
<li>The chances it is in your door is still 1/3.
<li>The chances it is not in your door is 2/3.
<li>The door opening part sets the odds for one door to 0, so that 2/3 is contained entirely in one door. 
</ul>

The entire point of this problem is to be unituitive, so having it be confusing is normal. 
<hr>

<h3>Exercise Scenario</h3>

Suppose you are placing a sports bet on your favorite team - The Bayes. You know a few things:
<ul>
<li>The Bayes have a 50% chance of winning a game. (Based on past performance)
<li>The Bayes have had a 10% chance of having rain in their games in Bayes Stadium.
<li>However, in games that The Bayes have won in Bayes Statium, there's be a 11% chance of rain. 
<li>
<li>P(W) = 50%
<li>P(R) = 10%
<li>P(R|W) = 11%
</ul>

<b>What is the probability that The Bayes win if it rains? </b>

In [ ]:
#Build Table
bet = pd.DataFrame(index=["Win", "Loss"])
bet["prior"] = Fraction(1,2)
bet

In [ ]:
#Add in the likelihoods.
#Given that we have a win/loss, how likely is rain. 
bet["likelihood"] = .11, .1
bet

In [ ]:
#Update to complete. 
update(bet)
bet